In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.insert(1, oj(sys.path[0], '..', 'vision_fit'))  # insert parent path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import math
# plt.style.use('dark_background')
from mog_fit import data
from collections import OrderedDict
from sklearn import preprocessing

import torch
from torch.autograd import Variable
from mog_analyze import viz
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

from vision_fit import data
import viz_weights
from process_results import process_results

%matplotlib inline
%load_ext autoreload
%autoreload 2

import style
cb = '#008fd5'
cr = '#fc4f30'

In [2]:
style.set_style()

# load results from a directory

In [3]:
# depending on how much is saved, this may take a while
out_dir = '/scratch/users/vision/yu_dl/raaz.rsk/track_acts/resweep_full_new'
fnames = sorted([fname for fname in os.listdir(out_dir) \
                 if not fname.startswith('weights') \
                 and not fname.startswith('idx')])

results_list = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in tqdm(fnames) 
                if not fname.startswith('weights') and not fname.startswith('idx')]
results = pd.concat(results_list, axis=1).T.infer_objects()

100%|██████████| 1306/1306 [00:16<00:00, 78.43it/s]


**add cols**

In [4]:
# adds these vec keys: fc0_fro, fc1_fro, fc0_stab_rank, fc1_stab_rank, act0_stab_rank, act1_stab_rank, corr0, corr1
# adds these scalar keys: max_train_acc, max_test_acc, _final of all the above
its, ts, results = process_results(results)
print(results.shape)

(1219, 84)


# plot max corrs
*note some things disappear in below plot as a result of nans*

In [ ]:
r = results
sweep_params = ['lr', 'optimizer', 'num_layers', 'dset', 'batch_size', 'shuffle_labels', 'hidden_size']# wanna group by everything but seed

r = r.groupby(sweep_params).mean().reset_index()
r = r[r.shuffle_labels == False]
r = r[r.num_layers == 4]
hue = 'optimizer'
cb = '#008fd5'
cr = '#fc4f30'

plt.figure(figsize=(7, 7))
R, C = 2, 2
rm = r[r.dset == 'mnist']
plt.subplot(R, C, 1)
plt.title('MNIST')
sns.scatterplot(rm.max_test_acc, rm.corr0_final, alpha=0.5)
plt.ylim(0, 1)
plt.xlabel('Test accuracy')
plt.ylabel('$\\rho_{mem}$ Final Layer 1')

plt.subplot(R, C, 2)
plt.title('MNIST')
sns.scatterplot(rm.max_test_acc, rm.corr1_final, alpha=0.5)
plt.ylim(0, 1)
plt.xlabel('Test accuracy')
plt.ylabel('$\\rho_{mem}$ Final Layer 2')


rm = r[r.dset == 'cifar10']
plt.subplot(R, C, 3)
plt.title('CIFAR')
sns.scatterplot(rm.max_test_acc, rm.corr0_final, alpha=0.5)
plt.ylim(0, 1)
plt.xlabel('Test accuracy')
plt.ylabel('$\\rho_{mem}$ Final Layer 1')


plt.subplot(R, C, 4)
plt.title('CIFAR')
sns.scatterplot(rm.max_test_acc, rm.corr1_final, alpha=0.5)
plt.ylim(0, 1)
plt.xlabel('Test accuracy')
plt.ylabel('$\\rho_{mem}$ Final Layer 2')

plt.tight_layout()
plt.savefig('generalization.pdf')
plt.show()

In [ ]:
r = results
r = r[r.shuffle_labels == False]
# r1 = r[r.optimizer == 'adam']
r = r[r.dset == 'cifar10']
# r = r[r.max_test_acc >= 0.4]
plt.plot(r.max_test_acc, r.max_train_acc - r.max_test_acc, 'o')
# plt.plot(r1.max_train_acc, r1.max_test_acc, 'o')